In [1]:
import pandas as pd
import numpy as np
from ta.momentum import rsi
from ta.utils import dropna
import datetime
from api_helper import ShoonyaApiPy
import logging
import sched, time


In [2]:
#enable dbug to see request and responses
logging.basicConfig(level=logging.DEBUG)

#start of our program
api = ShoonyaApiPy()

In [3]:
def loginme():
    #credentials
    uid    = 'FA54439'
    pwd     = 'Gurdeep#7966'
    factor2 = '23031990'
    vc      = 'FA54439_U'
    apikey  = 'a4c5620bebfb7be65eb2821deab0eb62'
    imei    = 'GuruPC'

    #make the api call
    login = api.login(userid=uid, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=apikey, imei=imei)
    if login['actid']=='FA54439':
        print('Login Success!!')
    else:
        print('Logout Failed!!')
    return True


In [4]:
def logoutme():
    logout = api.logout()
    if logout['stat']=='Ok':
        print('Logout Success!!')
    return True

In [5]:
def Supertrend(df, atr_period, multiplier):
    
    high = df['High']
    low = df['Low']
    close = df['Close']
    
    # calculate ATR
    price_diffs = [high - low, 
                   high - close.shift(), 
                   close.shift() - low]
    true_range = pd.concat(price_diffs, axis=1)
    true_range = true_range.abs().max(axis=1)
    # default ATR calculation in supertrend indicator
    atr = true_range.ewm(alpha=1/atr_period,min_periods=atr_period).mean() 
    # df['atr'] = df['tr'].rolling(atr_period).mean()
    
    # HL2 is simply the average of high and low prices
    hl2 = (high + low) / 2
    # upperband and lowerband calculation
    # notice that final bands are set to be equal to the respective bands
    final_upperband = upperband = hl2 + (multiplier * atr)
    final_lowerband = lowerband = hl2 - (multiplier * atr)
    
    # initialize Supertrend column to True
    supertrend = [True] * len(df)
    
    for i in range(1, len(df.index)):
        curr, prev = i, i-1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True
        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False
        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]
            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

        # to remove bands according to the trend direction
        if supertrend[curr] == True:
            final_upperband[curr] = np.nan
        else:
            final_lowerband[curr] = np.nan
    
    return pd.DataFrame({
        'Supertrend': supertrend,
        'Final Lowerband': final_lowerband,
        'Final Upperband': final_upperband
    }, index=df.index)

In [6]:
def searchsymbol(exch,scr):   
    exch  = exch #NSE,NFO,MCX etc
    query = scr #'BANKNIFTY 30DEC CE' # multiple criteria to narrow results 
    # ret = api.searchscrip(exchange=exch, searchtext=query)
    mytoken=api.searchscrip(exchange=exch, searchtext=query)['values'][0]['token']
    return mytoken

In [7]:
def getdataop(token,time):    
    lastBusDay = datetime.datetime.today()-datetime.timedelta(days=5)
    lastBusDay = lastBusDay.replace(hour=0, minute=0, second=0, microsecond=0)
    ret = api.get_time_price_series(exchange='NFO', token=token, starttime=lastBusDay.timestamp(), interval=time)
    df=pd.DataFrame(ret)
    df=df.iloc[::-1]
    df['time']=pd.to_datetime(df['time'].astype(str), format='%d-%m-%Y %H:%M:%S')
    df.set_index('time',inplace=True)
    df=df.iloc[:,2:]
    df=df.astype(float)
    # return only last 300 bars
    df=df.iloc[-300:]
    return df

In [8]:
def getdataeq(token,time):    
    lastBusDay = datetime.datetime.today()-datetime.timedelta(days=5)
    lastBusDay = lastBusDay.replace(hour=0, minute=0, second=0, microsecond=0)
    ret = api.get_time_price_series(exchange='NSE', token=token, starttime=lastBusDay.timestamp(), interval=time)
    df=pd.DataFrame(ret)
    df=df.iloc[::-1]
    df['time']=pd.to_datetime(df['time'].astype(str), format='%d-%m-%Y %H:%M:%S')
    df.set_index('time',inplace=True)
    df=df.iloc[:,2:]
    df=df.astype(float)
    # return only last 300 bars
    df=df.iloc[-300:]
    return df

In [9]:
###Get ITM CE and PE Strikes
def get_ITM_strike(df):
    spot=df.intc.iloc[-1:][0]
    atm=50 * round(int(spot)/50)
    itm_ce=atm-50
    itm_pe=atm+50
    return spot,itm_ce,itm_pe

In [10]:
####Get ITM CE Token
def get_option_token_ce(expiry,strike):
    cscr=f'NIFTY{expiry}22C{strike}'
    tokence=searchsymbol(exch='NFO',scr=cscr)
    return cscr,tokence

In [11]:
####Get ITM PE Token
def get_option_token_pe(expiry, strike):
    pscr=f'NIFTY{expiry}22P{strike}'
    tokenpe=searchsymbol(exch='NFO',scr=pscr)
    return pscr,tokenpe

In [12]:
def resampler(df,time):
    newdf=pd.DataFrame()
    newdf['Open']=df.into.resample(time).first()
    newdf['Close']=df.intc.resample(time).last()
    newdf['High']=df.inth.resample(time).max()
    newdf['Low']=df.intl.resample(time).min()
    newdf['Vwap']=df.intvwap.resample(time).last()
    newdf['Volume']=df.intv.resample(time).sum()
    newdf['OI']=df.intoi.resample(time).sum()
    # newdf.dropna(axis=0,inplace=True)
    return newdf

In [13]:
def predf(data):
    
    ###Base Time Frame########
    datadf=data.copy()
    
    datadf=resampler(datadf,'5min')
    supertrendBTF=Supertrend(datadf,10,1)
    supertrendBTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendBTF['rsima']=supertrendBTF['rsi'].rolling(13).mean()
    supertrendBTF.reset_index(inplace=True)

    ###High Time Frame#########
    datadf=data.copy()
    
    datadf=resampler(datadf,'15min')
    supertrendHTF=Supertrend(datadf,10,0.8)
    supertrendHTF['rsi']=rsi(datadf.Close,window=8,fillna=True)
    supertrendHTF['rsima']=supertrendHTF['rsi'].rolling(13).mean()
    supertrendHTF.reset_index(inplace=True)
    
    #######Merge Base and HTF indicators###
    st=pd.merge(supertrendBTF,supertrendHTF,on='time',how='left')
    st=st.replace(to_replace=np.nan,method='ffill')
    st=st.set_index('time')
    
    #########Join everything in single df#######
    datadf=data.copy()
    
    datadf=resampler(datadf,'5min')
    datadf=datadf.join(st)
    
    
    return datadf

In [14]:
loginme()

Login Success!!


True

In [15]:
# import time
# ce_buy_script=''
# pe_buy_script=''

# sec=True
# while sec==True:
#     print(float(str(datetime.datetime.now().time()).split(':')[2]))
#     starttime = time.time()
#     time.sleep(1.0 - ((time.time() - starttime) % 60.0))
#     if float(str(datetime.datetime.now().time()).split(':')[2])>58.0:
#         sec=False

# while True:
#     starttime = time.time()
#     time.sleep(62.0 - ((time.time() - starttime) % 60.0))
#     now=datetime.datetime.now().time()
#     if now >= datetime.time(9,20):
#         ## Get nifty token and Data to calculate ITM CE and PE Strikes
#         mytoken=searchsymbol('NSE','Nifty 50')
#         nifty=getdataeq(mytoken,1)
#         spot,itm_ce,itm_pe=get_ITM_strike(nifty)
#         print(f' Spot: {spot}, ITM Call: {itm_ce}, ITM PUT: {itm_pe}')

#         if not ce_buy_script:
#             ##Get ITM CE token and download the data
#             cscr,tokence=get_option_token_ce('17MAR',itm_ce)
#         if not pe_buy_script:
#             ##Get ITM PE token and download the data
#             pscr,tokenpe=get_option_token_pe('17MAR',itm_pe)


#         cedf=getdataop(token=tokence,time=1)
#         pedf=getdataop(token=tokenpe,time=1)

#         ##Preparing the Data before triggring actual Buy Sell Conditions 
#         cedf=predf(cedf)
#         pedf=predf(pedf)
#     print(f'Call {cedf.Close.tail(1)},Put {pedf.Close.tail(1)}')

In [32]:
import time
while True:
    mytoken=searchsymbol('NSE','Nifty 50')
    nifty=getdataeq(mytoken,1)
    spot,itm_ce,itm_pe=get_ITM_strike(nifty)
    print(f' Spot: {spot}, ITM Call: {itm_ce}')
    cscr,tokence=get_option_token_ce('17MAR',itm_ce)
    cedf=getdataop(token=tokence,time=15)
    print(cedf[-1:].intc)
    time.sleep(5)

 Spot: 16702.95, ITM Call: 16650
time
2022-03-15 13:45:00    211.1
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.0, ITM Call: 16650
time
2022-03-15 13:45:00    210.85
Name: intc, dtype: float64
 Spot: 16705.6, ITM Call: 16650
time
2022-03-15 13:45:00    

KeyboardInterrupt: 

In [65]:
mint=(4,9,14,19,24,29,34,39,44,49,54,59)
secs=(30)
current_minute=datetime.datetime.now().minute
current_minute=datetime.datetime.now().second
current_minute in mint

False

In [57]:
4 

True

In [18]:
datadf

,Open,Close,High,Low,Vwap,Volume,OI
time,,,,,,,
2022-03-10 09:15:00,160.00,237.25,316.90,160.00,246.97,176000.0,116100.0
2022-03-10 09:20:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2022-03-10 09:25:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2022-03-10 09:30:00,238.05,221.30,250.00,221.30,237.27,119800.0,22900.0
2022-03-10 09:35:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...
2022-03-15 12:40:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2022-03-15 12:45:00,258.20,244.60,262.45,236.65,249.93,127550.0,10100.0
2022-03-15 12:50:00,NaN,NaN,NaN,NaN,NaN,0.0,0.0


'2022-03-15 13:00:00'